In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import cv2

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
buddywatch = load_model('buddywatch_face.h5')

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120, 120))
    
    y_pred = buddywatch.predict(np.expand_dims(resized / 255,0))
    confidence = y_pred[0][0][0]
    bbox = y_pred[1][0]
    
    if confidence > 0.5:
        bbox_start = tuple(np.multiply(bbox[:2], [640, 480]).astype(int))
        bbox_end = tuple(np.multiply(bbox[2:], [640, 480]).astype(int))
        # Render the bounding box
        cv2.rectangle(frame, bbox_start, bbox_end, (255, 0, 0), 2)
        
        # Render rectangle for the label text
        label_box_start = tuple(np.add(np.multiply(bbox[:2], [640, 480]).astype(int), [0, -20]))
        label_box_end = tuple(np.add(np.multiply(bbox[:2], [640, 480]).astype(int), [110, 0]))
        cv2.rectangle(frame, label_box_start, label_box_end, (255, 0, 0), -1)
        
        # Render the label text
        position = tuple(np.add(np.multiply(bbox[:2], [640, 480]).astype(int), [0, -5]))
        text = f'buddy {confidence * 100:.1f}%'
        cv2.putText(
            frame,
            text,
            position,
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5, # Set font scaling
            (255, 255, 255), # Set color
            1, # Set font thickness
            cv2.LINE_AA
        )
    
    cv2.imshow('BuddyWatch', frame)

    # End if ESC is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break;
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step
